# **Pirate Pain Challenge — GRU Training**

Modello GRU monodirezionale per la classificazione dei soggetti `high_pain`, `low_pain`, `no_pain`, seguendo le analisi esplorative e le linee guida precedenti. Il notebook è pensato per Colab con GPU disponibile.


## 🌐 Google Drive Connection


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## ⚙️ Setup & Imports


In [ ]:
import os
import random
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from tqdm.auto import tqdm

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on device: {device}")


## ⏳ Data Loading


In [ ]:
DATA_DIR = Path('/content/drive/MyDrive/[2025-2026] AN2DL/Challenge')
DATA_PATH = DATA_DIR / 'data'

train_features_path = DATA_PATH / 'pirate_pain_train.csv'
train_labels_path = DATA_PATH / 'pirate_pain_train_labels.csv'
test_features_path = DATA_PATH / 'pirate_pain_test.csv'

print(train_features_path)

X_train_raw = pd.read_csv(train_features_path)
y_train = pd.read_csv(train_labels_path)
X_test_raw = pd.read_csv(test_features_path)

print('Train features:', X_train_raw.shape)
print('Train labels:', y_train.shape)
print('Test features:', X_test_raw.shape)
print('Label distribution:')
print(y_train['label'].value_counts())


## 🧼 Preprocessing


In [ ]:
def coerce_numeric_counts(df: pd.DataFrame) -> pd.DataFrame:
    word_to_num = {
        'zero': 0,
        'one': 1,
        'two': 2,
    }
    for col in ['n_legs', 'n_hands', 'n_eyes']:
        if col in df.columns:
            col_series = df[col].astype(str).str.lower()
            numeric_part = pd.to_numeric(col_series, errors='coerce')
            word_part = col_series.str.extract('(zero|one|two|three|four|five|six|seven|eight|nine)', expand=False)
            word_numeric = word_part.map(word_to_num)
            df[col] = numeric_part.fillna(word_numeric)
    return df

X_train_raw = coerce_numeric_counts(X_train_raw.copy())
X_test_raw = coerce_numeric_counts(X_test_raw.copy())

for df in (X_train_raw, X_test_raw):
    object_cols = df.select_dtypes(include='object').columns
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

numeric_cols = X_train_raw.select_dtypes(include=np.number).columns
print('Numeric feature count:', len(numeric_cols))


### Sequence Assembly


In [ ]:
FEATURES = [col for col in numeric_cols if col not in ['sample_index']]
TARGET_COLUMN = 'label'
SEQUENCE_LENGTH = X_train_raw.groupby('sample_index').size().iloc[0]
print('Sequence length:', SEQUENCE_LENGTH)
print('Number of features (including time):', len(FEATURES))

label_to_idx = {'no_pain': 0, 'low_pain': 1, 'high_pain': 2}
y_train['target'] = y_train[TARGET_COLUMN].map(label_to_idx)

assert not y_train['target'].isna().any(), 'Found unmapped labels.'


In [ ]:
def build_sequences(features_df: pd.DataFrame, label_df: pd.DataFrame | None = None,
                    feature_columns: List[str] | None = None,
                    sequence_length: int = SEQUENCE_LENGTH) -> Tuple[np.ndarray, np.ndarray | None, List[int]]:
    feature_columns = feature_columns or FEATURES
    sequences = []
    labels = []
    sample_ids = []

    grouped = features_df.groupby('sample_index')
    for sample_index, group in grouped:
        group = group.sort_values('time')
        values = group[feature_columns].values
        if len(values) != sequence_length:
            raise ValueError(f'Sequence {sample_index} has length {len(values)} != {sequence_length}')
        sequences.append(values)
        sample_ids.append(sample_index)
        if label_df is not None:
            target_value = label_df.loc[label_df['sample_index'] == sample_index, 'target'].iloc[0]
            labels.append(target_value)

    sequences = np.stack(sequences)
    labels = np.array(labels) if labels else None
    return sequences, labels, sample_ids

X_sequences, y_sequences, sample_ids = build_sequences(X_train_raw, y_train)
X_test_sequences, _, test_ids = build_sequences(X_test_raw, label_df=None)

print('Sequences shape:', X_sequences.shape)
print('Test sequences shape:', X_test_sequences.shape)


### Train/Validation Split


In [ ]:
train_idx, val_idx, y_train_split, y_val_split = train_test_split(
    np.arange(len(X_sequences)),
    y_sequences,
    test_size=0.2,
    random_state=SEED,
    stratify=y_sequences,
)

X_train_seq = X_sequences[train_idx]
y_train_seq = y_sequences[train_idx]
X_val_seq = X_sequences[val_idx]
y_val_seq = y_sequences[val_idx]

print('Train sequences:', X_train_seq.shape)
print('Validation sequences:', X_val_seq.shape)


### Normalisation


In [ ]:
train_flat = X_train_seq.reshape(-1, X_train_seq.shape[-1])
feature_mean = train_flat.mean(axis=0)
feature_std = train_flat.std(axis=0) + 1e-8

X_train_norm = (X_train_seq - feature_mean) / feature_std
X_val_norm = (X_val_seq - feature_mean) / feature_std
X_test_norm = (X_test_sequences - feature_mean) / feature_std

print('Normalisation applied. Mean close to 0:', np.abs(X_train_norm.mean()).round(4))


## 📦 PyTorch Dataset & Dataloaders


In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, sequences: np.ndarray, labels: np.ndarray | None = None):
        self.sequences = torch.tensor(sequences, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long) if labels is not None else None

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        x = self.sequences[idx]
        if self.labels is None:
            return x
        y = self.labels[idx]
        return x, y

train_dataset = SequenceDataset(X_train_norm, y_train_seq)
val_dataset = SequenceDataset(X_val_norm, y_val_seq)
test_dataset = SequenceDataset(X_test_norm)

class_counts = np.bincount(y_train_seq)
class_weights = class_counts.sum() / (len(class_counts) * class_counts)
print('Class counts:', class_counts)
print('Class weights:', class_weights.round(3))

sample_weights = class_weights[y_train_seq]
weighted_sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=weighted_sampler, drop_last=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

input_features = X_train_norm.shape[-1]
print('Input features for GRU:', input_features)


## 🧠 GRU Model


In [ ]:
class MonoGRU(nn.Module):
    def __init__(self, input_size: int, hidden_size: int = 128, num_layers: int = 2, dropout: float = 0.2, num_classes: int = 3):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
            bidirectional=False,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        gru_out, hidden = self.gru(x)
        last_hidden = hidden[-1]
        out = self.dropout(last_hidden)
        logits = self.fc(out)
        return logits

model = MonoGRU(input_size=input_features, hidden_size=160, num_layers=2, dropout=0.3, num_classes=len(label_to_idx)).to(device)
print(model)

params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {params/1e6:.2f}M')


## 🏋️ Training Utilities


In [ ]:
@dataclass
class TrainConfig:
    epochs: int = 120
    patience: int = 15
    lr: float = 1e-3
    weight_decay: float = 1e-4


def compute_class_weights(labels: np.ndarray, base_weights: np.ndarray) -> torch.Tensor:
    weights = torch.tensor(base_weights, dtype=torch.float32, device=device)
    return weights


def train_one_epoch(model: nn.Module, dataloader: DataLoader, criterion, optimizer) -> Tuple[float, float]:
    model.train()
    total_loss = 0.0
    preds = []
    targets = []

    for batch in dataloader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        preds.append(logits.argmax(dim=1).detach().cpu().numpy())
        targets.append(labels.detach().cpu().numpy())

    avg_loss = total_loss / len(dataloader.dataset)
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    macro_f1 = classification_report(targets, preds, output_dict=True)['macro avg']['f1-score']
    return avg_loss, macro_f1


def evaluate(model: nn.Module, dataloader: DataLoader, criterion) -> Tuple[float, float, np.ndarray, np.ndarray]:
    model.eval()
    total_loss = 0.0
    preds = []
    targets = []

    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            logits = model(inputs)
            loss = criterion(logits, labels)

            total_loss += loss.item() * inputs.size(0)
            preds.append(logits.argmax(dim=1).cpu().numpy())
            targets.append(labels.cpu().numpy())

    avg_loss = total_loss / len(dataloader.dataset)
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    macro_f1 = classification_report(targets, preds, output_dict=True)['macro avg']['f1-score']
    return avg_loss, macro_f1, preds, targets


def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, class_weights: torch.Tensor, config: TrainConfig):
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_f1 = -np.inf
    best_state = None
    history = {'train_loss': [], 'val_loss': [], 'train_f1': [], 'val_f1': []}
    patience_counter = 0

    for epoch in range(1, config.epochs + 1):
        train_loss, train_f1 = train_one_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_f1, _, _ = evaluate(model, val_loader, criterion)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)

        print(f"Epoch {epoch:03d} | Train Loss {train_loss:.4f} F1 {train_f1:.4f} || Val Loss {val_loss:.4f} F1 {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= config.patience:
                print('Early stopping triggered.')
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return history, best_f1


## 🚀 Training


In [ ]:
config = TrainConfig(epochs=120, patience=20, lr=3e-4, weight_decay=1e-4)
class_weight_tensor = compute_class_weights(y_train_seq, class_weights)

history, best_val_f1 = train_model(model, train_loader, val_loader, class_weight_tensor, config)
print(f'Best validation macro F1: {best_val_f1:.4f}')


## 📈 Validation Metrics


In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weight_tensor)
val_loss, val_f1, val_preds, val_targets = evaluate(model, val_loader, criterion)
print(f'Validation loss: {val_loss:.4f} | macro F1: {val_f1:.4f}')

idx_to_label = {v: k for k, v in label_to_idx.items()}
print('\nClassification report:')
print(classification_report(val_targets, val_preds, target_names=[idx_to_label[i] for i in range(len(idx_to_label))]))

cm = confusion_matrix(val_targets, val_preds)
cm_df = pd.DataFrame(cm, index=[idx_to_label[i] for i in range(len(idx_to_label))], columns=[idx_to_label[i] for i in range(len(idx_to_label))])
cm_df


### Loss & Macro-F1 Curves


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Validation')
axes[0].set_title('Loss')
axes[0].set_xlabel('Epoch')
axes[0].legend()

axes[1].plot(history['train_f1'], label='Train')
axes[1].plot(history['val_f1'], label='Validation')
axes[1].set_title('Macro F1')
axes[1].set_xlabel('Epoch')
axes[1].legend()

plt.tight_layout()
plt.show()


## 💾 Save Model & Artifacts


In [ ]:
artifacts_dir = DATA_DIR / 'artifacts'
artifacts_dir.mkdir(exist_ok=True)

model_path = artifacts_dir / 'gru_monodirectional.pt'
scaler_path = artifacts_dir / 'feature_normalisation.npz'
label_map_path = artifacts_dir / 'label_mapping.json'

torch.save({'state_dict': model.state_dict(), 'config': vars(config)}, model_path)
np.savez(scaler_path, mean=feature_mean, std=feature_std, features=FEATURES)

import json
with open(label_map_path, 'w') as f:
    json.dump({str(k): int(v) for k, v in label_to_idx.items()}, f)

print('Saved model to:', model_path)
print('Saved scaler to:', scaler_path)
print('Saved label map to:', label_map_path)


## 🧪 Test Set Inference (Optional)


In [ ]:
model.eval()
test_preds = []
with torch.no_grad():
    for batch in test_loader:
        inputs = batch.to(device)
        logits = model(inputs)
        test_preds.append(logits.argmax(dim=1).cpu().numpy())

test_preds = np.concatenate(test_preds)
submission = pd.DataFrame({
    'sample_index': test_ids,
    'label': [idx_to_label[int(idx)] for idx in test_preds]
})
submission_path = artifacts_dir / 'gru_predictions.csv'
submission.to_csv(submission_path, index=False)
print('Saved submission predictions to:', submission_path)


## ✅ Next Steps

- Monitorare il recall della classe `high_pain`: se resta basso, valutare focal loss o oversampling mirato sui soggetti più difficili.
- Introdurre augmentation temporale leggera (rumore gaussiano controllato, time warping moderato) per aumentare la varietà senza distruggere i pattern utili.
- Effettuare hyperparameter search su `hidden_size`, `num_layers` e `dropout` mantenendo il vincolo GRU monodirezionale.


## ✅ Next Steps

- Monitor `high_pain` recall: se resta bassa, considerare focal loss o oversampling mirato sui soggetti con peggiore performance.
- Integrare augmentation temporale (noise, time warping leggero) rispettando i pattern individuati nell'EDA.
- Valutare fine-tuning di `hidden_size`, `num_layers` e `dropout` per migliorare la generalizzazione.
